In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats
from scipy.stats import norm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [5]:
import os
path = './auxiliary-data/auxiliary-data'
auxdata_namelist = os.listdir(path)
print(auxdata_namelist)

['sg-gov-markets-hawker-centres.csv', 'sg-commerical-centres.csv', 'sg-primary-schools.csv', 'sg-shopping-malls.csv', 'sg-secondary-schools.csv', 'sg-train-stations.csv']


#  Famous Primary School

In [7]:
from geopy.distance import geodesic

famous_pschl = ['Nanyang Primary School','Catholic High School (Primary)','Tao Nan School','Nan Hua Primary School','St. Hilda\'s Primary School','Henry Park Primary School','Anglo-Chinese School (Primary)','Raffles Girls\' Primary School','Pei Hwa Presbyterian Primary School','CHIJ St Nicholas Girls\' School']


In [8]:
auxiliary_df_pschl = pd.read_csv(os.path.join(path,'sg-primary-schools.csv'))
pschl_famous_df = auxiliary_df_pschl[ auxiliary_df_pschl['name'].isin(famous_pschl)]
pschl_famous_df.head()

,name,lat,lng
7,Anglo-Chinese School (Primary),1.318752,103.835077
23,Catholic High School (Primary),1.354928,103.844988
70,Henry Park Primary School,1.317057,103.784216
101,Nan Hua Primary School,1.306517,103.764771
102,Nanyang Primary School,1.320832,103.808217


In [9]:
pschl_famous_ary = np.array(pschl_famous_df[['lat','lng']])
def famous_pschl_distance(lat,lng):
    dist_ = []
    for i in pschl_famous_ary:
        dist_.append(geodesic((lat,lng),i).km)
    
    return sum(d<=2 for d in dist_)

In [10]:
train_df['no_of_nearest_famous_pshcl'] = train_df.apply(lambda row: famous_pschl_distance(row['lat'],row['lng']),axis=1)
train_df['no_of_nearest_famous_pshcl'].value_counts()

0    17483
1     7427
2      822
3      316
Name: no_of_nearest_famous_pshcl, dtype: int64

In [11]:
test_df['no_of_nearest_famous_pshcl'] = test_df.apply(lambda row: famous_pschl_distance(row['lat'],row['lng']),axis=1)
test_df['no_of_nearest_famous_pshcl'].value_counts()

0    4979
1    2199
2     244
3      78
Name: no_of_nearest_famous_pshcl, dtype: int64

# define the acceptable distance

In [12]:
fac_tolerant_dist = {'sg-commerical-centres.csv': 3,
                          'sg-gov-markets-hawker-centres.csv': 0.5,
                          'sg-train-stations.csv': 1,
                          'sg-shopping-malls.csv': 3,
                          'sg-primary-schools.csv': 1.5,
                          'sg-secondary-schools.csv': 3}

def cal_dist(lat,lng,fac_loc_ary,tolerant_dist):
    dist_ = []
    for i in fac_loc_ary:
        dist_.append(geodesic((lat,lng),i).km)
    
    return sum(d<=tolerant_dist for d in dist_)

for fac_csvname in fac_tolerant_dist.keys():
    print("="*30,fac_csvname,"="*30)
    facility_df = pd.read_csv(os.path.join(path,fac_csvname))
    fac_loc_ary = np.array(facility_df[['lat','lng']])
    row_name = fac_csvname.split(".")[0].split('-')[1]
    train_df['no_of_nearest_'+row_name] = train_df.apply(lambda row: cal_dist(row['lat'],row['lng'],fac_loc_ary,fac_tolerant_dist[fac_csvname]),axis=1)
    test_df['no_of_nearest_'+row_name] = test_df.apply(lambda row: cal_dist(row['lat'],row['lng'],fac_loc_ary,fac_tolerant_dist[fac_csvname]),axis=1)
    print("-----train_df------")
    print(train_df['no_of_nearest_'+row_name].value_counts())
    print("-----test_df------")
    print(test_df['no_of_nearest_'+row_name].value_counts())       


============================== sg-commerical-centres.csv ==============================
-----train_df------
2    7615
3    6843
1    5928
0    2714
4    1101
7     814
5     556
6     477
Name: no_of_nearest_commerical, dtype: int64
-----test_df------
2    2162
3    1945
1    1769
0     787
4     329
7     219
5     156
6     133
Name: no_of_nearest_commerical, dtype: int64
============================== sg-gov-markets-hawker-centres.csv ==============================
-----train_df------
0    18884
1     5507
2     1414
3      129
4      110
5        4
Name: no_of_nearest_gov, dtype: int64
-----test_df------
0    5450
1    1587
2     385
4      38
3      35
5       5
Name: no_of_nearest_gov, dtype: int64
============================== sg-train-stations.csv ==============================
-----train_df------
1     7291
2     7250
0     4660
3     2389
6     1652
5      901
4      653
7      513
9      465
8      235
10      39
Name: no_of_nearest_train, dtype: int64
-----test_df------
2 

In [14]:
train_df.to_csv('./train_add_aux.csv',index=False)
test_df.to_csv('./test_add_aux.csv',index=False)